In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
from collections import defaultdict
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any

In [2]:
class ModelConf(NamedTuple):
    name: str
    directory: Path
    model_max_length: int=512
    batch_size: int=16
    weight: float=1
        

class Conf(NamedTuple):
    debug: bool = False  #not os.getenv('KAGGLE_IS_COMPETITION_RERUN', False)
    input_dir: Path = Path("/kaggle/input")
    comp_dir: Path = input_dir / "pii-detection-removal-from-educational-data"
    temp_dir: Path = Path('/kaggle/temp')
    # write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
    working_dir: Path = Path('/kaggle/working')
    resource_dir: Path = input_dir / "libtlalpii/tlal-pii-0.1"
    data_dir: Path = resource_dir / "input"
    models: List[ModelConf] = [
        ModelConf(
            name="deberta_v3_base",
            directory=Path(resource_dir) / "models/ner/deberta_v3_base/20240420_185849",
            model_max_length=512,
            batch_size=32,
            weight=0.5,
        ),
        ModelConf(
            name="deberta_v3_large",
            directory=Path(resource_dir) / "models/ner/deberta_v3_large/20240421_072931",
            model_max_length=512,
            batch_size=32,
            weight=0.5,
        ),
    ]
    window_length: int = 512
    window_stride: int = 256
    outside_label_threshold: float = 0.80
    duplicate_dt_strategy: str = "first"  # first, mean, min_outside_proba 

conf = Conf()
print(conf)

Conf(debug=False, input_dir=PosixPath('/kaggle/input'), comp_dir=PosixPath('/kaggle/input/pii-detection-removal-from-educational-data'), temp_dir=PosixPath('/kaggle/temp'), working_dir=PosixPath('/kaggle/working'), resource_dir=PosixPath('/kaggle/input/libtlalpii/tlal-pii-0.1'), data_dir=PosixPath('/kaggle/input/libtlalpii/tlal-pii-0.1/input'), models=[ModelConf(name='deberta_v3_base', directory=PosixPath('/kaggle/input/libtlalpii/tlal-pii-0.1/models/ner/deberta_v3_base/20240420_185849'), model_max_length=512, batch_size=32, weight=0.5), ModelConf(name='deberta_v3_large', directory=PosixPath('/kaggle/input/libtlalpii/tlal-pii-0.1/models/ner/deberta_v3_large/20240421_072931'), model_max_length=512, batch_size=32, weight=0.5)], window_length=512, window_stride=256, outside_label_threshold=0.8, duplicate_dt_strategy='first')


In [3]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, Tesla P100-PCIE-16GB
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [4]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
sys.path.append(str(conf.input_dir / "sgcharts-ml/src"))
sys.path.append(str(conf.resource_dir / "src"))
import scml
from scml import nlp as snlp
from scml import pandasx as pdx
from mylib.ner import predict_ner_proba, NerDataset, blend_predictions
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
scml.seed_everything()

In [5]:
fp = conf.comp_dir / "test.json"
if conf.debug:
    fp = conf.comp_dir / "train.json"
with open(str(fp)) as f:
    data = json.load(f)
print(f"len(data)={len(data)}")
texts: List[List[str]] = []
dids: List[str] = []
for row in data:
    texts.append(row["tokens"])
    dids.append(str(row["document"]))

len(data)=10


# Model Inference

In [6]:
ds = NerDataset(
    tokenizer=None,
    texts=texts,
    document_ids=dids,
    window_length=conf.window_length,
    window_stride=conf.window_stride,
)
dwm_map=defaultdict(list)
for m, mc in enumerate(conf.models):
    print(mc.name)
    tokenizer = AutoTokenizer.from_pretrained(
        str(mc.directory),
        model_max_length=mc.model_max_length,
    )
    ds.tokenizer = tokenizer
    model = AutoModelForTokenClassification.from_pretrained(str(mc.directory))
    # (seqs, sequence length in tokens, classes)
    y_proba = predict_ner_proba(
        ds=ds,
        model=model,
        batch_size=mc.batch_size,
        device=device,
    )
    print(f"{y_proba.shape} y_proba[0][0]={y_proba[0][0]}")
    # token-to-word mapping (document, word): (models, classes)
    for i in range(len(y_proba)):
        d=int(ds.document_ids[i])
        for j in range(len(y_proba[i])):
            w = ds.word_ids[i][j]
            if w is None:
                continue
            w+=ds.word_ranges[i][0]
            # collect all predictions including duplicate doc-word pairs
            if conf.duplicate_dt_strategy=="first" and len(dwm_map[(d,w,m)])!=0:
                continue
            dwm_map[(d,w,m)].append(y_proba[i][j])
dw_map=defaultdict(list)
for k,v in dwm_map.items():
    d,w,m = k
    if conf.duplicate_dt_strategy=="min_outside_proba":
        i = np.argmin(v, axis=0)[0]  # `Outside` label at index 0
        dw_map[(d,w)].append(v[i].flatten().tolist())
    else:
        dw_map[(d,w)].append(np.mean(v, axis=0).flatten().tolist()) 
del dwm_map
gc.collect()

deberta_v3_base


/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


(38, 512, 15) y_proba[0][0]=[0.07480966 0.06523772 0.0831486  0.06141015 0.06134651 0.07266962
 0.06743815 0.05385804 0.06865056 0.05009668 0.07634687 0.04750584
 0.10992236 0.05568699 0.05187228]
deberta_v3_large
(38, 512, 15) y_proba[0][0]=[0.0854906  0.18612398 0.02502726 0.05608674 0.05952589 0.1378629
 0.02561215 0.04259685 0.04575884 0.06643269 0.04495351 0.08801769
 0.03996282 0.01869089 0.07785716]


90

In [7]:
del texts, dids, ds, model, tokenizer
gc.collect()

0

In [8]:
sub = blend_predictions(
    weights=np.array([[mc.weight for mc in conf.models]], dtype=np.float32),
    dw_map=dw_map,
    outside_label_threshold=conf.outside_label_threshold,
)
sub.to_csv("submission.csv", index=False)
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  34 non-null     int64 
 1   token     34 non-null     int64 
 2   label     34 non-null     object
 3   row_id    34 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.2+ KB


In [9]:
sub.head(10)

,document,token,label,row_id
0,7,9,B-NAME_STUDENT,0
1,7,10,I-NAME_STUDENT,1
2,7,482,B-NAME_STUDENT,2
3,7,483,I-NAME_STUDENT,3
4,7,738,B-NAME_STUDENT,4
5,7,741,B-NAME_STUDENT,5
6,7,742,I-NAME_STUDENT,6
7,10,0,B-NAME_STUDENT,7
8,10,1,I-NAME_STUDENT,8
9,10,464,B-NAME_STUDENT,9


# Debug

In [10]:
#!python -V && which python
#!pip list